In [9]:
from schrodinger.application.desmond.packages import traj_util, traj, topo
#from schrodinger.application.desmond.packages import traj_util, traj, topo
import numpy as np
import pandas as pd

import os
cms_path = os.path.expanduser('~/Dropbox/KcsA/kcsa_5000ns_1218/kcsa_deactivated_5us_centered-out.cms')
traj_path = os.path.expanduser('~/Dropbox/KcsA/kcsa_5000ns_1218/kcsa_deactivated_5us_centered_trj')

# Load structure and trajectory (paths to your .cms and .trj files)
_, cms_model = topo.read_cms(cms_path)
trajectory = traj.read_traj(traj_path)

# Select backbone atoms: carbonyl carbon (C), oxygen (O), and amide nitrogen (N)
c_aids = cms_model.select_atom("chain.name A and res.num 21 and atom.name C")
o_aids = cms_model.select_atom("chain.name A and res.num 76 and atom.name O")
n_aids = cms_model.select_atom("chain.name A and res.num 76 and atom.name N")

print(c_aids)

# Map atom IDs to global IDs for trajectory processing
gid_C_list = topo.aids2gids(cms_model, c_aids)
gid_O_list = topo.aids2gids(cms_model, o_aids)
gid_N_list = topo.aids2gids(cms_model, n_aids)

# Ensure the lists are of equal length (one-to-one correspondence)
assert len(gid_C_list) == len(gid_O_list) == len(gid_N_list), "Mismatch in atom selections."

# Initialize lists to accumulate orientation vectors over time
sigma_33_list, sigma_22_list, sigma_11_list = [], [], []

# # Loop through trajectory frames
# for frame in trajectory:
#     # Process each residue (assume one-to-one correspondence of C, O, N per residue)
#     for gid_C, gid_O, gid_N in zip(gid_C_list, gid_O_list, gid_N_list):
#         # Extract atomic positions
#         C = frame.pos(gid_C)[0]
#         O = frame.pos(gid_O)[0]
#         N = frame.pos(gid_N)[0]
        
#         # Build local molecular frame
#         vec_CO = O - C
#         sigma_22 = vec_CO / np.linalg.norm(vec_CO)  # Along CO bond

#         # Normal vector to peptide plane (C', O, N)
#         vec_CN = N - C
#         sigma_33 = np.cross(vec_CO, vec_CN)  # Perpendicular to peptide plane
#         sigma_33 /= np.linalg.norm(sigma_33)

#         # Orthogonal axis
#         sigma_11 = np.cross(sigma_33, sigma_22)  # Orthogonal to sigma_33 and sigma_22
#         sigma_11 /= np.linalg.norm(sigma_11)

#         # Append to orientation lists
#         sigma_33_list.append(sigma_33)
#         sigma_22_list.append(sigma_22)
#         sigma_11_list.append(sigma_11)

# # Post-processing: Save or analyze the accumulated orientation data
# sigma_33_array = np.array(sigma_33_list)
# sigma_22_array = np.array(sigma_22_list)
# sigma_11_array = np.array(sigma_11_list)

print(sigma_33_array.shape)



frame_times = np.array([fr.time for fr in trajectory])  # Extract time for each frame in the trajectory

# # Create a DataFrame
# df = pd.DataFrame({
#     'Frame': np.arange(len(frame_times)),  # Frame indices
#     'Time (ps)': frame_times,             # Trajectory time
#     'Sigma_33_X': sigma_33_array[:,0],   # X-component of Sigma_33
#     'Sigma_33_Y': sigma_33_array[1],   # Y-component of Sigma_33
#     'Sigma_33_Z': sigma_33_array[2],   # Z-component of Sigma_33
#     'Sigma_22_X': sigma_22_array[0],   # X-component of Sigma_22
#     'Sigma_22_Y': sigma_22_array[1],   # Y-component of Sigma_22
#     'Sigma_22_Z': sigma_22_array[2],   # Z-component of Sigma_22
#     'Sigma_11_X': sigma_11_array[0],   # X-component of Sigma_11
#     'Sigma_11_Y': sigma_11_array[1],   # Y-component of Sigma_11
#     'Sigma_11_Z': sigma_11_array[2],   # Z-component of Sigma_11
# })

# # Save the DataFrame to an Excel file
# output_file = "orientation_vectors.xlsx"
# df.to_excel(output_file, index=False)

# print(f"Orientation data saved to {output_file}")




[]
(0,)


In [38]:
test=cms_model.select_atom('chain.name A and res.num 1')
for i in test:
    print(cms_model.atom[i].name)
#len(cms_model.chain['A'].residue)

N768
C769
C770
O771










In [23]:
cms_model.atom[1156]

In [2]:
import numpy as np

# Example: use u_z_list from above to build A(t)
u_z_arr = np.array(u_z_list)            # shape (Nframes,3)
cos2 = u_z_arr[:,2]**2                 # z-component squared
A = (3*cos2 - 1) * (Delta_sigma) / 2.0  # CSA interaction sequence

# Autocorrelation (unnormalized, one-sided)
N = len(A)
A_mean = np.mean(A)
acf = np.array([np.mean((A[:N-k]-A_mean)*(A[k:]-A_mean)) for k in range(N)])


2
